In [11]:
import pyodbc,pymysql,pandas as pd

from sqlalchemy import create_engine

In [12]:
#connecting to Microsoft sql server database
connection_params = {
    'driver': 'SQL Server',
    'server': 'DESKTOP-K6HKGAC',
    'database': 'CustomInfo1',
    'trusted_connection': 'yes',  }

# Construct the connection string to the single line 
conn_str1 = ';'.join([f'{key}={value}' for key, value in connection_params.items()])


In [13]:
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str1}')

#  SQL query used to get all datarom the table CustomerData
mssql_query = "SELECT * FROM dbo.CustomerData"

# Use read_sql with the engine
try:
    with engine.connect() as connection:
        mssql_df = pd.read_sql(mssql_query, connection)
    print(mssql_df)
except Exception as e:
    print(f"Error: {e}")

   ClientNumber  BranchCode CustomerType Title  \
0          3621          24            I   Mr.   
1          3627          24            I   Mr.   
2          3739          24            I   Mr.   
3          3798          24            I   Mr.   
4          3804          24            I   Mr.   
5          5399          24            I   Mr.   
6         54594          24            I   Mr.   
7         93421          24            I   Mr.   
8        103971          24            I  Mrs.   
9        181920          24            I   Mr.   

                                                Name  
0                    XXXXXXXXX XXXXXXXX XXXXXXXXXXXX  
1    XXXXXXXXXX XXXXXXXXXXXX XXXXXXXX XXXXXX  XXXXXX  
2  XXXXXX XXXXXXXXXXXXX XXXXXXX XXX XXXXXX XXXXXX...  
3             XXXXXX XXXXXXXXXXXXX XXXXXXXX  XXXXXXX  
4    XXXXXXXXXX XXXXXXXXXXXXX XXXXXXXX  XXXXXXXXXXXX  
5             XXXXXXXXXXXX XXXXXXXX XXXXXXXX  XXXXXX  
6                    XXXXXXXXXXXX  XXXXX  XXXXXXXXXX  
7        

In [14]:
maria_conn_params ={
    'host':'localhost',
    'user':'Dinuka',
    'password':'1QXlmCFrDEJgZXHx',
    'database':'custominfo2',
}


In [16]:
try:
    mariadb_conn = pymysql.connect(**maria_conn_params)
    print("Connected to MariaDB!")

    # Your MariaDB table query
    mariadb_query = "SELECT * FROM customerinfo"

    # Use pandas to read data from MariaDB
    mariadb_df = pd.read_sql(mariadb_query, mariadb_conn)
    print(mariadb_df)

except Exception as e:
    print(f"Error: {e}")

finally:
    mariadb_conn.close()

Connected to MariaDB!
    ClientNumber CustomerCategory CustomerSubCategory  SegmentCode  \
0           3621               01                0102            4   
1           3627               01                0102            4   
2           3739               01                0102            4   
3           3798               01                0102            4   
4         197611               01                0102            1   
5         197614               01                0102            1   
6         197618               01                0102            1   
7         197630               01                0102            1   
8         197635               01                0102            1   
9         197665               01                0102            1   
10        197671               01                0102            1   

    BusinessDivision OpeningDate TaxCode  
0                  4  2015-07-02    TDS9  
1                  4  2015-07-02    TDS9  
2       

C:\Users\Theijan\AppData\Local\Temp\ipykernel_1268\515581028.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mariadb_df = pd.read_sql(mariadb_query, mariadb_conn)


In [17]:
inner_df = pd.merge(mariadb_df, mssql_df, how='inner', left_on='ClientNumber', right_on='ClientNumber')

left_df = pd.merge(mariadb_df, mssql_df, how='left', left_on='ClientNumber', right_on='ClientNumber')

right_df = pd.merge(mariadb_df, mssql_df, how='right', left_on='ClientNumber', right_on='ClientNumber')

outer_df = pd.merge(mariadb_df, mssql_df, how='outer', left_on='ClientNumber', right_on='ClientNumber')

union_df = pd.concat([mariadb_df,mssql_df], ignore_index=True)


In [18]:
print(inner_df)

   ClientNumber CustomerCategory CustomerSubCategory  SegmentCode  \
0          3621               01                0102            4   
1          3627               01                0102            4   
2          3739               01                0102            4   
3          3798               01                0102            4   

   BusinessDivision OpeningDate TaxCode  BranchCode CustomerType Title  \
0                 4  2015-07-02    TDS9          24            I   Mr.   
1                 4  2015-07-02    TDS9          24            I   Mr.   
2                 4  2015-07-03    TDS9          24            I   Mr.   
3                 4  2015-07-06    TDS9          24            I   Mr.   

                                                Name  
0                    XXXXXXXXX XXXXXXXX XXXXXXXXXXXX  
1    XXXXXXXXXX XXXXXXXXXXXX XXXXXXXX XXXXXX  XXXXXX  
2  XXXXXX XXXXXXXXXXXXX XXXXXXX XXX XXXXXX XXXXXX...  
3             XXXXXX XXXXXXXXXXXXX XXXXXXXX  XXXXXXX  


In [19]:
connection_params = {
    'driver': 'SQL Server',
    'server': 'DESKTOP-K6HKGAC',
    'database': 'Joined_customer_database',
    'trusted_connection': 'yes',  }

# Construct the connection string to the single line 
conn_str1 = ';'.join([f'{key}={value}' for key, value in connection_params.items()])


In [20]:
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str1}')

try:
    inner_df.to_sql('inner_table', con=engine, index=False, if_exists='replace')
    left_df.to_sql('left_table', con=engine, index=False, if_exists='replace')
    right_df.to_sql('right_table', con=engine, index=False, if_exists='replace')
    outer_df.to_sql('outer_table', con=engine, index=False, if_exists='replace')
    union_df.to_sql('union_table',con=engine, index=False, if_exists='replace')
    
    print("Tables successfully saved to SQL Server!")

except Exception as e:
    print(f"Error: {e}")

finally:
    engine.dispose() 

Tables successfully saved to SQL Server!
